## **Install the Libraries**

In [10]:
!pip install langchain_community langchain_groq langchain_huggingface

## **Import the API KEY**

In [11]:
from google.colab import userdata
key = userdata.get('groq')

## **Initialize LLM and embeddings**

In [12]:
import os
import getpass
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

os.environ["GROQ_API_KEY"] =key
llm = ChatGroq(model="llama3-8b-8192")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore

# Load the dataset
file_path = "/content/drive/MyDrive/mobile_packages.csv"
loader = CSVLoader(file_path=file_path)
docs = loader.load()
# print(docs)

# Split the Document
text_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
splits = text_spliter.split_documents(docs)

# Create an in-memory vector store
vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding=embeddings)

retriever = vectorstore.as_retriever()

In [24]:
from langchain_core.prompts import PromptTemplate

# Define prompt template
prompt_template = PromptTemplate(
    template="""
    You are an assistant that helps with mobile packages. Use the following retrieved documents to answer the question:
    {context}

    Question: {question}

    Answer:
    """,
    input_variables=["context", "question"]
)

## **Build a QA RAG chain**

In [27]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)

## **Example Query**

In [28]:
# Example query
query = "What is the best package for Pubg ?"
response = qa_chain.invoke({"query": query})
print("Response:", response)

Response: {'query': 'What is the best package for Pubg ?', 'result': 'Based on the provided documents, I would recommend the "Zong Monthly PUBG" package for playing PUBG. This package has a data limit of 10GB, which is significantly higher than the data limits of the Telenor daily packages. The Telenor daily packages have a data limit of 50MB, which may not be sufficient for playing PUBG, especially if you play for extended periods or have a high-quality connection.\n\nAdditionally, the Zong Monthly PUBG package is specifically designed for PUBG players, which suggests that it may be optimized for the game\'s requirements. While the Telenor daily packages are also internet-based, they are more general-purpose bundles that may not be tailored specifically for gaming.\n\nOverall, the Zong Monthly PUBG package appears to be the best option for PUBG players, given its higher data limit and potential optimization for gaming.'}
